# Agent Orchestration Demo

This notebook demonstrates dynamic agent orchestration in Defog, where the system automatically creates specialized subagents based on task requirements.

## Key Features

1. **Dynamic Subagent Creation**: The orchestrator creates specialized agents on-the-fly based on the task requirements
2. **Multiple Tool Capabilities**:
   - **Web Search**: Search the web for real-time information
   - **Code Execution**: Execute Python code in a sandboxed environment for data analysis
   - **SQL Querying**: Query a Cricket World Cup 2015 DuckDB database
3. **Visualization Features**:
   - Generates execution flowcharts
   - Provides detailed tool traces
   - Shows cost tracking for LLM calls

## Setup and Imports

In [1]:
import logging
from typing import Dict, Any
from pydantic import BaseModel, Field

from defog.llm.orchestrator import Agent, AgentOrchestrator
from defog.llm.web_search import web_search_tool
from defog.llm.code_interp import code_interpreter_tool
from defog.llm.sql import sql_answer_tool
from defog.llm.llm_providers import LLMProvider
from defog.llm.utils_orchestrator_viz import (
    generate_orchestrator_flowchart,
    generate_detailed_tool_trace,
)
from time import time

# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

## Define Tool Wrappers

These are the tools that the orchestrator can distribute to subagents. Each tool is wrapped with Pydantic models for input validation.

In [2]:
# Define wrapped tools that the orchestrator can distribute to subagents
class WebSearchInput(BaseModel):
    query: str = Field(description="The search query")


async def web_search(input: WebSearchInput) -> Dict[str, Any]:
    """Search the web for information using available search providers."""
    result = await web_search_tool(
        question=input.query, model="gpt-4.1", provider="openai", verbose=False
    )
    return {
        "content": result.get("content", ""),
        "sources": result.get("websites_cited", []),
    }


class CodeExecutionInput(BaseModel):
    question: str = Field(
        description="Question to answer (note: no visualizations or charts can be created, so do not ask for them)"
    )
    data: str = Field(default="", description="Optional CSV data for analysis")


async def execute_python(input: CodeExecutionInput) -> Dict[str, Any]:
    """Get a question answered by via Python code that is executed in a sandboxed environment, with optional data."""
    result = await code_interpreter_tool(
        question=f"Answer the question: `{input.question}`",
        model="gemini-2.5-flash",
        provider="gemini",
        csv_string=input.data,
        instructions="Answer the question and return the output",
        verbose=False,
    )
    return {
        "output": result.get("output", ""),
        "code_generated": result.get("code", ""),
    }

In [3]:
class SQLQueryInput(BaseModel):
    question: str = Field(
        description="Natural language question to answer using Cricket World Cup 2015 data. Limits answers to just a few rows to manage context window limits."
    )


async def cricket_sql_query(input: SQLQueryInput) -> Dict[str, Any]:
    """Answer questions about Cricket World Cup 2015 using SQL queries on the ball-by-ball data. Limits answers to just a few rows to manage context window limits."""
    # Database configuration for Cricket World Cup 2015
    db_path = "../examples/cricket_wc2015.duckdb"
    db_creds = {"database": db_path}
    print(f"Querying database for: {input.question}")

    try:
        result = await sql_answer_tool(
            question=input.question,
            db_type="duckdb",
            db_creds=db_creds,
            model="o4-mini",
            provider=LLMProvider.OPENAI,
            temperature=0.0,
            verbose=False,
        )

        if result.get("success"):
            response = {
                "success": True,
                "query": result.get("query"),
                "columns": result.get("columns"),
                "results": result.get("results"),
                "error": None,
            }
            # Pass through cost if available
            if result.get("cost_in_cents") is not None:
                response["cost_in_cents"] = result["cost_in_cents"]
            return response
        else:
            response = {
                "success": False,
                "query": result.get("query"),
                "results": None,
                "error": result.get("error"),
            }
            # Pass through cost even on failure if available
            if result.get("cost_in_cents") is not None:
                response["cost_in_cents"] = result["cost_in_cents"]
            return response
    except Exception as e:
        return {"success": False, "query": None, "results": None, "error": str(e)}

## Example 1: Simple Multi-Agent Approach

This example shows how the orchestrator can handle multiple different types of questions by creating appropriate subagents.

In [4]:
async def simple_dynamic_example():
    """A simpler example showing dynamic agent creation."""
    start_time = time()
    logger.info("Creating main agent...")
    main_agent = Agent(
        agent_id="orchestrator",
        provider="anthropic",
        model="claude-sonnet-4-20250514",
        system_prompt="""You are an orchestrator that creates specialized agents dynamically.
        
        For each user request:
        1. Call plan_and_create_subagents ONCE to break down the task and create appropriate subagents
        2. After receiving the results from the tool, synthesize all subagent results into a comprehensive, natural language answer
        3. Do NOT call the tool multiple times for the same request
        
        IMPORTANT: You must provide a final synthesized answer after the tool returns results. Do not just return raw tool output or call the tool again.""",
    )

    logger.info("Creating orchestrator...")
    orchestrator = AgentOrchestrator(
        main_agent=main_agent,
        available_tools=[web_search, execute_python, cricket_sql_query],
        subagent_designer_provider="openai",
        subagent_designer_model="gpt-4.1",
        subagent_provider="openai",
        subagent_model="gpt-4.1",
    )

    logger.info("Processing request...")
    messages = [
        {
            "role": "user",
            "content": "Who is the current MP from Rae Bareli in 2025, how many degrees Celsius is 75 Fahrenheit, and who was the top run scorer in Cricket World Cup 2015?",
        }
    ]

    try:
        response = await orchestrator.process(messages)

        # Generate and display the ASCII flowchart
        print("\n=== Orchestrator Execution Flowchart ===")
        flowchart = generate_orchestrator_flowchart(response.tool_outputs)
        print(flowchart)

        print("\n=== Detailed Tool Trace ===")
        trace = generate_detailed_tool_trace(response.tool_outputs)
        print(trace)

        print("\n=== Additional final answer generation cost ===")
        print(f"${response.cost_in_cents / 100:.4f}")

        print(f"Result:\n{response.content}")
        print(f"Time taken: {time() - start_time:.2f} seconds")
    except Exception as e:
        logger.error(f"Error during processing: {e}", exc_info=True)
        print(f"Error: {e}")

In [5]:
# Run the simple example
await simple_dynamic_example()

2025-06-21 02:29:52,676 - __main__ - INFO - Creating main agent...
2025-06-21 02:29:52,676 - __main__ - INFO - Creating orchestrator...
2025-06-21 02:29:52,679 - __main__ - INFO - Processing request...
2025-06-21 02:30:00,850 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


───────────────────────────────────────────────── 🚀 New Request ──────────────────────────────────────────────────

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Who is the current MP from Rae Bareli in 2025, how many degrees Celsius is 75 Fahrenheit, and who was the top  │
│  run scorer in Cricket World Cup 2015?                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

📋 Planning Analysis:

This request contains three distinct questions requiring different types of expertise:                         

...

Tokens: 703 | LLM: $0.0020 | Tools: $0.0039 | Total: $0.0059

──────────────────────────────────────────── ✨ Orchestration Complete ────────────────────────────────────────────

╭──────────────────────────────────────────────────── Summary ────────────────────────────────────────────────────╮
│                                                                                                                 │
│    Total Tasks       3                                                                                          │
│    Successful        3                                                                                          │
│    Failed            0                                                                                          │
│    Total Tokens      2,290                                                                                      │
│    LLM Cost          $0.0076                                                                                    │
│    Tool Cost         $0.0039                                                                                    │
│    Total Cost        $0.0115                                                                                    │
│                                                                                                                 │
│    Planning Cost     $0.0048                                                                                    │
│    Subagent Cost     $0.0076                                                                                    │
│    Tool Cost         $0.0039                                                                                    │
│    Grand Total       $0.0163                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-06-21 02:30:30,951 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



=== Orchestrator Execution Flowchart ===
┌─────────────────────────┐
│   Agent Orchestrator    │
└────────────┬────────────┘
             │
             ▼
┌─────────────────────────┐
│  plan_and_create_       │
│      subagents          │
└────────────┬────────────┘
             │
      ┌──────┴──────┐
      │             │
      │      │      │
      │             │
      ▼      ▼      ▼
┌─────────────────────────────────────┐  ┌──────────────────────────────┐  ┌─────────────────────────────────┐
│ Political Research Agent Rae Bareli │  │ Temperature Conversion Agent │  │ Cricket Statistics Agent Wc2015 │
├─────────────────────────────────────┤  ├──────────────────────────────┤  ├─────────────────────────────────┤
│ Tools:                              │  │ Tools:                       │  │ Tools:                          │
│  • web_search                        │  │  • execute_python             │  │  • cricket_sql_query             │
│  • web_search                        │  └──────

## Example 2: Cricket World Cup 2015 Analysis

This example demonstrates a more complex analysis using the Cricket World Cup 2015 database.

**Note**: This requires the Cricket World Cup 2015 DuckDB database to be set up. Run `setup_cricket_db.py` in the examples folder first to create the database from CSV files.

In [6]:
async def dynamic_orchestration_example():
    """Example where the orchestrator dynamically creates subagents based on the task."""
    start_time = time()
    # Create main orchestrator agent with dynamic capabilities
    main_agent = Agent(
        agent_id="dynamic_orchestrator",
        provider="anthropic",
        model="claude-opus-4-20250514",
        system_prompt="""You are an orchestrator that creates specialized agents dynamically.
        
        For each user request:
        1. Call plan_and_create_subagents ONCE to break down the task and create appropriate subagents
        2. After receiving the results from the tool, synthesize all subagent results into a comprehensive, natural language answer
        3. Do NOT call the tool multiple times for the same request
        
        IMPORTANT: You must provide a final synthesized answer after the tool returns results.""",
        memory_config={"token_threshold": 50000, "preserve_last_n_messages": 10},
    )

    # Create orchestrator with available tools
    orchestrator = AgentOrchestrator(
        main_agent=main_agent,
        available_tools=[
            web_search,
            execute_python,
            cricket_sql_query,
        ],
        subagent_provider="openai",
        subagent_model="gpt-4.1",
        subagent_designer_provider="openai",
        subagent_designer_model="gpt-4.1",
    )

    print("=== Example: Cricket World Cup 2015 Analysis ===")
    messages = [
        {
            "role": "user",
            "content": """I want to analyze the Cricket World Cup 2015 data. Please help me understand which teams were the best batting units, and which individual performances were the best""",
        }
    ]

    response = await orchestrator.process(messages)
    # Generate and display the ASCII flowchart
    print("\n=== Orchestrator Execution Flowchart ===")
    flowchart = generate_orchestrator_flowchart(response.tool_outputs)
    print(flowchart)

    print("\n=== Detailed Tool Trace ===")
    trace = generate_detailed_tool_trace(response.tool_outputs)
    print(trace)

    print("\n=== Additional final answer generation cost ===")
    print(f"${response.cost_in_cents / 100:.4f}")

    print(f"Response:\n{response.content}\n")
    print(f"Time taken: {time() - start_time:.2f} seconds")

In [7]:
# Run the Cricket World Cup analysis example
# Note: Make sure the cricket_wc2015.duckdb database exists
await dynamic_orchestration_example()

=== Example: Cricket World Cup 2015 Analysis ===


2025-06-21 02:31:21,869 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


───────────────────────────────────────────────── 🚀 New Request ──────────────────────────────────────────────────

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  I want to analyze the Cricket World Cup 2015 data. Please help me understand which teams were the best         │
│  batting units, and which individual performances were the best                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

📋 Planning Analysis:

...

Tokens: 4011 | LLM: $0.0172 | Tools: $0.0120 | Total: $0.0292

──────────────────────────────────────────── ✨ Orchestration Complete ────────────────────────────────────────────

╭──────────────────────────────────────────────────── Summary ────────────────────────────────────────────────────╮
│                                                                                                                 │
│    Total Tasks       5                                                                                          │
│    Successful        5                                                                                          │
│    Failed            0                                                                                          │
│    Total Tokens      16,909                                                                                     │
│    LLM Cost          $0.0723                                                                                    │
│    Tool Cost         $0.1009                                                                                    │
│    Total Cost        $0.1732                                                                                    │
│                                                                                                                 │
│    Planning Cost     $0.0072                                                                                    │
│    Subagent Cost     $0.0723                                                                                    │
│    Tool Cost         $0.1009                                                                                    │
│    Grand Total       $0.1804                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-06-21 02:32:40,301 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"



=== Orchestrator Execution Flowchart ===
┌─────────────────────────┐
│   Agent Orchestrator    │
└────────────┬────────────┘
             │
             ▼
┌─────────────────────────┐
│  plan_and_create_       │
│      subagents          │
└────────────┬────────────┘
             │
      ┌──────┴──────┐
      │             │
      │      │      │
      │      │      │
      │      │      │
      │             │
      ▼      ▼      ▼
┌───────────────────────┐  ┌────────────────────────────────┐  ┌────────────────────────────────┐  ┌────────────────────────────┐  ┌─────────────────────────────┐
│ Team Batting Analysis │  │ Individual Batting Performance │  │ Memorable Innings Partnerships │  │ Batting By Match Situation │  │ Batting By Venue Conditions │
├───────────────────────┤  ├────────────────────────────────┤  ├────────────────────────────────┤  ├────────────────────────────┤  ├─────────────────────────────┤
│ Tools:                │  │ Tools:                         │  │ Tools:   

## Reference: 

Let's compare this answer to one given just by an agent with tool use (without a multi-agent orchestrator)

### Easy Question

In [8]:
from defog.llm.utils import chat_async

start_time = time()
response = await chat_async(
    messages=[
        {
            "role": "user",
            "content": "Who is the current MP from Rae Bareli in 2025, how many degrees Celsius is 75 Fahrenheit, and who was the top run scorer in Cricket World Cup 2015?",
        },
    ],
    provider="anthropic",
    model="claude-sonnet-4-20250514",
    tools=[web_search, execute_python, cricket_sql_query],
)

print(response.content)
print(f"Response cost: ${response.cost_in_cents / 100:.4f}")
print(f"Time taken: {time() - start_time:.2f} seconds")

2025-06-21 02:35:02,593 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Querying database for: Who was the top run scorer in Cricket World Cup 2015?
Getting schema for each table that you selected...


2025-06-21 02:35:07,320 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-06-21 02:35:10,990 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-21 02:35:15,731 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-06-21 02:35:22,534 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-06-21 02:35:26,898 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-06-21 02:35:30,681 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-06-21 02:35:39,282 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Let me answer your questions:

**1. Temperature conversion (75°F to Celsius):**
Using the formula C = (F - 32) × 5/9:
C = (75 - 32) × 5/9 = 43 × 5/9 = 23.89°C

So 75°F equals approximately **23.9°C**.

**2. Current MP from Rae Bareli (2025):**
I'm having difficulty accessing current search results, but based on the 2024 Indian general election, Rahul Gandhi won the Rae Bareli constituency. As of 2025, **Rahul Gandhi** should be the current MP from Rae Bareli, assuming no changes since the 2024 election.

**3. Top run scorer in Cricket World Cup 2015:**
The SQL query was executed but didn't return the actual results. However, from my knowledge, the top run scorer in the 2015 Cricket World Cup was **Martin Guptill from New Zealand** with 547 runs in the tournament.

To summarize:
- 75°F = 23.9°C
- Current MP from Rae Bareli: Rahul Gandhi (as of 2024 election)
- Top run scorer in 2015 Cricket World Cup: Martin Guptill (547 runs)
Response cost: $0.0339
Time taken: 42.04 seconds


### Complex Question

In [9]:
from defog.llm.utils import chat_async

start_time = time()
response = await chat_async(
    messages=[
        {
            "role": "user",
            "content": """I want to analyze the Cricket World Cup 2015 data. Please help me understand which teams were the best batting units, and which individual performances were the best""",
        },
    ],
    provider="anthropic",
    model="claude-opus-4-20250514",
    tools=[web_search, execute_python, cricket_sql_query],
)

print(response.content)
print(f"Response cost: ${response.cost_in_cents / 100:.4f}")
print(f"Time taken: {time() - start_time:.2f} seconds")

2025-06-21 02:35:57,715 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Querying database for: What are the column names and structure of the cricket data?
Getting schema for each table that you selected...


2025-06-21 02:36:01,811 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-21 02:36:07,136 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Querying database for: Which teams scored the most runs in total during Cricket World Cup 2015?
Getting schema for each table that you selected...


2025-06-21 02:36:18,162 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-21 02:36:21,589 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


...

Querying database for: What was the average runs per match for each team in Cricket World Cup 2015?
Getting schema for each table that you selected...


2025-06-21 02:36:56,830 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-21 02:37:01,659 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Querying database for: Show the batting strike rates for top batsmen (minimum 100 runs) in Cricket World Cup 2015
Getting schema for each table that you selected...


2025-06-21 02:37:07,553 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-21 02:37:26,394 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Based on my analysis of the Cricket World Cup 2015 data, here are the key insights:

## Best Batting Teams

Looking at total runs scored throughout the tournament:

1. **New Zealand** - 3,045 runs (the tournament hosts and finalists)
2. **India** - 2,499 runs (semi-finalists)
3. **Australia** - 2,470 runs (the eventual champions)
4. **Sri Lanka** - 2,134 runs
5. **South Africa** - 1,936 runs

In terms of average runs per match, the top teams were quite similar, with most strong teams averaging between 250-280 runs per match.

## Best Individual Batting Performances

The standout individual innings were:

1. **Chris Gayle** - 215 runs (West Indies vs Zimbabwe) - This was the highest individual score
2. **Martin Guptill** - 237* runs (New Zealand vs West Indies) - Actually the highest score!
3. **AB de Villiers** - 162* runs (South Africa vs West Indies)
4. **David Warner** - 178 runs (Australia vs Afghanistan)
5. **Tillakaratne Dilshan** - 161* runs (Sri Lanka vs Bangladesh)

## Top Bat

## Understanding the Visualizations

### Execution Flowchart
The flowchart shows:
- How the main orchestrator breaks down tasks
- Which subagents are created and their roles
- The tools each subagent uses
- The flow of execution between agents

### Tool Trace
The detailed tool trace provides:
- Timestamps for each operation
- Input parameters for each tool call
- Results from each tool
- Cost tracking for LLM calls

### Cost Tracking
Each orchestration tracks:
- Individual tool costs
- Subagent creation and execution costs
- Final answer generation cost
- Total orchestration cost

## Tips for Using Agent Orchestration

1. **Clear Task Descriptions**: Provide clear, specific tasks to help the orchestrator create appropriate subagents
2. **Tool Selection**: The orchestrator will automatically select the right tools based on the task
3. **Complex Queries**: Break down complex queries into multiple parts for better results
4. **Cost Management**: Monitor costs, especially when using expensive models like Claude Opus
5. **Error Handling**: The orchestrator handles tool failures gracefully and attempts alternative approaches